In [0]:
dbutils.fs.mkdirs("dbfs:/FileStore/Formula1/qualifying")

Out[1]: True

In [0]:
#Step 1 - Read the JSON file using the spark dataframe reader API

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [0]:

qualifying_schema = StructType(fields=[StructField("qualifyId", IntegerType(), False),
                                      StructField("raceId", IntegerType(), True),
                                      StructField("driverId", IntegerType(), True),
                                      StructField("constructorId", IntegerType(), True),
                                      StructField("number", IntegerType(), True),
                                      StructField("position", IntegerType(), True),
                                      StructField("q1", StringType(), True),
                                      StructField("q2", StringType(), True),
                                      StructField("q3", StringType(), True),
                                     ])

In [0]:
qualifying_df = spark.read.json("dbfs:/FileStore/Formula1/qualifying",schema =qualifying_schema , multiLine= True)

In [0]:
qualifying_df.show()

+---------+------+--------+-------------+------+--------+--------+--------+--------+
|qualifyId|raceId|driverId|constructorId|number|position|      q1|      q2|      q3|
+---------+------+--------+-------------+------+--------+--------+--------+--------+
|        1|    18|       1|            1|    22|       1|1:26.572|1:25.187|1:26.714|
|        2|    18|       9|            2|     4|       2|1:26.103|1:25.315|1:26.869|
|        3|    18|       5|            1|    23|       3|1:25.664|1:25.452|1:27.079|
|        4|    18|      13|            6|     2|       4|1:25.994|1:25.691|1:27.178|
|        5|    18|       2|            2|     3|       5|1:25.960|1:25.518|1:27.236|
|        6|    18|      15|            7|    11|       6|1:26.427|1:26.101|1:28.527|
|        7|    18|       3|            3|     7|       7|1:26.295|1:26.059|1:28.687|
|        8|    18|      14|            9|     9|       8|1:26.381|1:26.063|1:29.041|
|        9|    18|      10|            7|    12|       9|1:26.919

In [0]:
#Step 2 - Rename columns and add new columns
#Rename qualifyingId, driverId, constructorId and raceId
#Add ingestion_date with current timestamp

In [0]:
from pyspark.sql.functions import lit , current_timestamp

In [0]:
final_df = qualifying_df.withColumnRenamed("qualifyId", "qualify_id") \
.withColumnRenamed("driverId", "driver_id") \
.withColumnRenamed("raceId", "race_id") \
.withColumnRenamed("constructorId", "constructor_id") \
.withColumn("ingestion_date", current_timestamp()) 

In [0]:
final_df.show(truncate=False)

+----------+-------+---------+--------------+------+--------+--------+--------+--------+-----------------------+
|qualify_id|race_id|driver_id|constructor_id|number|position|q1      |q2      |q3      |ingestion_date         |
+----------+-------+---------+--------------+------+--------+--------+--------+--------+-----------------------+
|1         |18     |1        |1             |22    |1       |1:26.572|1:25.187|1:26.714|2024-04-23 04:00:32.158|
|2         |18     |9        |2             |4     |2       |1:26.103|1:25.315|1:26.869|2024-04-23 04:00:32.158|
|3         |18     |5        |1             |23    |3       |1:25.664|1:25.452|1:27.079|2024-04-23 04:00:32.158|
|4         |18     |13       |6             |2     |4       |1:25.994|1:25.691|1:27.178|2024-04-23 04:00:32.158|
|5         |18     |2        |2             |3     |5       |1:25.960|1:25.518|1:27.236|2024-04-23 04:00:32.158|
|6         |18     |15       |7             |11    |6       |1:26.427|1:26.101|1:28.527|2024-04-

In [0]:
#Step 3 - Write to output to processed container in parquet format

In [0]:
final_df.write.parquet("dbfs:/FileStore/Formula1/processed/f1_processed.qualifying") 

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-3417363049632157>:1
----> 1 final_df.write.parquet("dbfs:/FileStore/Formula1/processed/f1_processed.qualifying")

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1655, in DataFrameWriter.parquet(self, path, mode, partitionBy, compression)
   1653     self.partitionBy(partitionBy)
   1654 self._set_opts(compression=compression)
-> 1655 self._jwrite.parquet(path)

File /databricks/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py:1321, in Ja

In [0]:
final_df.show(truncate=False)

+----------+-------+---------+--------------+------+--------+--------+--------+--------+-----------------------+
|qualify_id|race_id|driver_id|constructor_id|number|position|q1      |q2      |q3      |ingestion_date         |
+----------+-------+---------+--------------+------+--------+--------+--------+--------+-----------------------+
|1         |18     |1        |1             |22    |1       |1:26.572|1:25.187|1:26.714|2024-04-23 04:01:10.743|
|2         |18     |9        |2             |4     |2       |1:26.103|1:25.315|1:26.869|2024-04-23 04:01:10.743|
|3         |18     |5        |1             |23    |3       |1:25.664|1:25.452|1:27.079|2024-04-23 04:01:10.743|
|4         |18     |13       |6             |2     |4       |1:25.994|1:25.691|1:27.178|2024-04-23 04:01:10.743|
|5         |18     |2        |2             |3     |5       |1:25.960|1:25.518|1:27.236|2024-04-23 04:01:10.743|
|6         |18     |15       |7             |11    |6       |1:26.427|1:26.101|1:28.527|2024-04-